## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-26-04-01-05 +0000,nyt,Trump’s Trip to Scotland Echoes an Earlier Vis...,https://www.nytimes.com/2025/07/26/world/europ...
1,2025-07-26-04-01-00 +0000,wsj,The Ukrainian Police Agency at the Center of P...,https://www.wsj.com/world/europe/ukraine-nabu-...
2,2025-07-26-04-00-00 +0000,wsj,Washington Struggles to Rein In an Emboldened ...,https://www.wsj.com/world/middle-east/washingt...
3,2025-07-26-03-44-01 +0000,bbc,UK condemns Hong Kong cash offer for help in a...,https://www.bbc.com/news/articles/cdx069we39xo
4,2025-07-26-03-43-46 +0000,nyt,"Cleo Laine, Grammy-Winning Jazz Singer With a ...",https://www.nytimes.com/2025/07/25/arts/music/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,51
86,new,19
98,gaza,17
96,will,12
59,out,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
182,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo,110
165,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...,96
52,2025-07-25-23-06-53 +0000,wapo,"Amid starvation scenes in Gaza, Trump administ...",https://www.washingtonpost.com/national-securi...,89
213,2025-07-25-13-23-00 +0000,wsj,Trump Administration Sues New York City Over S...,https://www.wsj.com/us-news/trump-administrati...,88
138,2025-07-25-18-31-45 +0000,nypost,Elon Musk hits back at Trump after president s...,https://nypost.com/2025/07/25/business/elon-mu...,85


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
182,110,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo
102,71,2025-07-25-20-45-00 +0000,wsj,Rep. Alexandria Ocasio-Cortez of New York stil...,https://www.wsj.com/politics/policy/aoc-met-ga...
36,46,2025-07-26-00-25-07 +0000,nyt,Thailand Warns of War With Cambodia as Deadly ...,https://www.nytimes.com/2025/07/25/world/asia/...
165,45,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...
93,44,2025-07-25-21-10-00 +0000,cbc,France plans to recognize Palestinian statehoo...,https://www.cbc.ca/news/politics/france-macron...
62,38,2025-07-25-22-30-20 +0000,nypost,Southwest flight plummets nearly 500 feet ‘in ...,https://nypost.com/2025/07/25/us-news/southwes...
29,35,2025-07-26-01-00-00 +0000,wsj,The Democratic Party’s image has eroded to its...,https://www.wsj.com/politics/elections/democra...
292,33,2025-07-25-04-17-55 +0000,nypost,George Santos bids dramatic farewell to suppor...,https://nypost.com/2025/07/25/us-news/george-s...
141,32,2025-07-25-18-18-54 +0000,wapo,Russia rules out Putin-Zelensky meeting until ...,https://www.washingtonpost.com/world/2025/07/2...
48,29,2025-07-25-23-24-00 +0000,wsj,HHS Secretary RFK Jr. plans to remove all memb...,https://www.wsj.com/health/healthcare/rfk-heal...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
